In [1]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5EncoderModel
from torch import nn
import torch
from torch import optim

from Datasets import Modes, WikiTable
from PIL import Image

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

ds = WikiTable(Modes.TRAIN, tokenizer=tokenizer, img_shape=(500, 500))

In [3]:
# pergunta = "what was the last year where this team was a part of the usl a-league?"
# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# target = tokenizer.encode_plus(pergunta,
#                                        padding='max_length',
#                                        truncation=True,
#                                        max_length=128,
#                                        return_tensors='pt')

In [4]:
# encoder = T5EncoderModel.from_pretrained('t5-base')
# decoder = decoder = T5ForConditionalGeneration.from_pretrained('t5-base')

In [5]:
# decoder.config.d_ff

In [6]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                                   nn.BatchNorm2d(out_channels),
                                   nn.LeakyReLU(),
                                   nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=2, padding=1),
                                   nn.BatchNorm2d(out_channels),
                                   nn.LeakyReLU())

    def forward(self, x):
        return self.block(x)

embedding_extractor = nn.Sequential(ConvBlock(3, 16),
                                     ConvBlock(16, 64),
                                     ConvBlock(64, 256),
                                     ConvBlock(256, 768))

In [7]:
sample = ds[5]

In [8]:
sample

{'table_img': tensor([[[-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          ...,
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059]],
 
         [[-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          ...,
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059]],
 
         [[-0.9059, -0.9059, -0.9059,  ..., -0.9059, -0.9059, -0.9059],
          [-0.9

In [9]:
patches = embedding_extractor(ds[0]['table_img'].unsqueeze(0)).shape

In [10]:
import itertools

In [11]:

def sample():
    encoder = T5EncoderModel.from_pretrained('t5-base')
    decoder = T5ForConditionalGeneration.from_pretrained('t5-base')
    sample = ds[5]
    
#     enc_out = encoder(input_ids=sample['question_ids'].unsqueeze(0),
#                      attention_mask=sample['question_attn_mask'].unsqueeze(0))
    
#     hidden = enc_out.last_hidden_state
    
#     print(f"hidden shp: {hidden.shape}")
    
#     print(sample['question_ids'].shape, sample['question_attn_mask'].shape)
    
    #torch.Size([1, 768, 32, 32])
#     features = embedding_extractor(ds[0]['table_img'].unsqueeze(0))
    
    #torch.Size([1, 32, 32, 768]) -> torch.Size([1, 32*32, 768])
#     features = features.permute(0, 2, 3, 1).view(1, -1, decoder.config.d_model)
    
#     torch.Size([1, 1152, 768])
#     features = torch.cat([features, hidden], dim =1)
    
#     print(f"feat shape: {features.shape}")
    
    #torch.Size([1, 768, 1152])
#     features = features.permute(0, 2, 1)
    
    linear = nn.Linear(1152, 128)
    
    #torch.Size([1, 768, 128])
#     proj = linear(features)
    
    #hidden state
    #torch.Size([1, 128, 728])
#     proj = proj.permute(0, 2, 1)
    
    #batch size x seqlen x self.d_model
#     print(f"proj shape: {proj.shape}")
    
    opt = optim.Adam(itertools.chain(encoder.parameters(), decoder.parameters(),
                                    linear.parameters(), embedding_extractor.parameters()), lr=1e-4)
    
    for _ in range(30):
        opt.zero_grad()
        
        encoder.train()
        decoder.train()
        
        
        enc_out = encoder(input_ids=sample['question_ids'].unsqueeze(0),
                     attention_mask=sample['question_attn_mask'].unsqueeze(0))
        
        hidden = enc_out.last_hidden_state
        features = embedding_extractor(sample['table_img'].unsqueeze(0))
        features = features.permute(0, 2, 3, 1).view(1, -1, decoder.config.d_model)
        features = torch.cat([features, hidden], dim =1)
        features = features.permute(0, 2, 1)
        
        proj = linear(features)
        proj = proj.permute(0, 2, 1)
        
        loss = decoder(encoder_outputs=(proj, ), 
                       labels=sample['target_ids'].unsqueeze(0)).loss
        
        print(f"Vai pfv: {loss}")
        loss.backward()
        opt.step()
        
        with torch.no_grad():
            encoder.eval()
            decoder.eval()
            embedding_extractor.eval()
        
            
            enc_out = encoder(input_ids=sample['question_ids'].unsqueeze(0),
                     attention_mask=sample['question_attn_mask'].unsqueeze(0))
        
            encoder_hidden_states = enc_out.last_hidden_state
            features = embedding_extractor(sample['table_img'].unsqueeze(0))
            features = features.permute(0, 2, 3, 1).view(1, -1, decoder.config.d_model)
            features = torch.cat([features, encoder_hidden_states], dim =1)
            features = features.permute(0, 2, 1)

            proj = linear(features)
            
            #-------------------------------------------------
            encoder_hidden_states = proj.permute(0, 2, 1)
#             print(f"Hidden shp: {encoder_hidden_states.shape}")
            
            decoded_ids = torch.full((1, 1),
                                 decoder.config.decoder_start_token_id,
                                 dtype=torch.long)
            
            for step in range(20):
                outputs = decoder(decoder_input_ids=decoded_ids,
                                  encoder_outputs=(encoder_hidden_states,),
                                  return_dict=True)
                logits = outputs["logits"]

                next_token_logits = logits[:, -1, :]

                # Greedy decoding
                next_token_id = next_token_logits.argmax(1).unsqueeze(-1)

                # Check if output is end of senquence for all batches
                if torch.eq(next_token_id[:, -1], tokenizer.eos_token_id).all():
                    break

                # Concatenate past ids with new id, keeping batch dimension
                decoded_ids = torch.cat([decoded_ids, next_token_id], dim=-1)
            
            print(f"Decoded: {tokenizer.batch_decode(decoded_ids, skip_special_tokens=True)}")
            print(f"Real: {sample['answer']}")
            
    del encoder, decoder, hidden, enc_out, features, linear, proj, opt

sample()

Some weights of the model checkpoint at t5-base were not used when initializing T5EncoderModel: ['decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.b

Vai pfv: 10.959039688110352
Decoded: ['a a solitary a single a single.']
Real: Lake Palas Tuzla
Vai pfv: 10.099553108215332
Decoded: ['a a solitary a single a single.']
Real: Lake Palas Tuzla
Vai pfv: 8.624160766601562
Decoded: ['a a solitary a single a single a']
Real: Lake Palas Tuzla
Vai pfv: 9.869156837463379
Decoded: ['a a solitary a single a single a']
Real: Lake Palas Tuzla
Vai pfv: 7.795677185058594
Decoded: ['a a solitary a single a single a']
Real: Lake Palas Tuzla
Vai pfv: 8.571985244750977
Decoded: ['a a a solitary a single a single']
Real: Lake Palas Tuzla
Vai pfv: 6.2537150382995605
Decoded: ['a a a solitary a single a single']
Real: Lake Palas Tuzla
Vai pfv: 7.2113261222839355
Decoded: ['a a a solitary a single a single']
Real: Lake Palas Tuzla
Vai pfv: 6.357486724853516
Decoded: ['a a a solitary a single a single']
Real: Lake Palas Tuzla
Vai pfv: 5.1611328125
Decoded: ['a a a sandbox a sandbox']
Real: Lake Palas Tuzla
Vai pfv: 5.619405269622803
Decoded: ['']
Real: Lake 

In [ ]:
##### encoder = T5EncoderModel.from_pretrained('t5-base')

In [120]:
target.input_ids.shape, target.attention_mask.shape

(torch.Size([1, 128]), torch.Size([1, 128]))

In [ ]:
decoder.get_encoder()

In [121]:
a = encoder(input_ids=target.input_ids, attention_mask=target.attention_mask, output_attentions=True)

In [122]:
a.last_hidden_state

tensor([[[-0.2139,  0.1358,  0.2902,  ..., -0.5094, -0.5432, -0.0968],
         [-0.2147, -0.0238, -0.1447,  ..., -0.2354, -0.1443, -0.3103],
         [-0.0410,  0.0897, -0.3892,  ..., -0.2407, -0.1751, -0.6424],
         ...,
         [-0.4393,  0.1653, -0.1272,  ...,  0.0248,  0.2123,  0.0096],
         [-0.4393,  0.1653, -0.1272,  ...,  0.0248,  0.2123,  0.0096],
         [-0.4393,  0.1653, -0.1272,  ...,  0.0248,  0.2123,  0.0096]]],
       grad_fn=<MulBackward0>)

In [ ]:
a

In [124]:
v.shape, a.last_hidden_state.shape

(torch.Size([1, 1536, 16, 40]), torch.Size([1, 128, 768]))

In [125]:
test = torch.cat([v.view(1, -1),a.last_hidden_state.view(1, -1)], dim=1)

In [126]:
test.shape

torch.Size([1, 1081344])

In [64]:
labels = tokenizer.encode_plus("204",
                                       padding='max_length',
                                       truncation=True,
                                       max_length=128,
                                       return_tensors='pt').input_ids

In [132]:
labels.shape

torch.Size([1, 128])

In [20]:
decoder = T5ForConditionalGeneration.from_pretrained('t5-base')

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [168]:
decoder(encoder_outputs=(t.permute(0, 2, 1), ), labels=labels).loss

tensor(9.3917, grad_fn=<NllLossBackward>)

In [67]:
decoder.config.d_model

768

In [148]:
ajustado = embedding_extractor(img_tensor.unsqueeze(0)).permute(0, 2, 3, 1)

In [149]:
ajustado = ajustado.view(1, -1, 768)

torch.Size([1, 25, 63, 768])

In [150]:
ajustado = ajustado.view(1, -1, 768)

In [151]:
ajustado.shape

torch.Size([1, 1575, 768])

In [159]:
ajustado.permute(0, 2, 1).shape

torch.Size([1, 768, 1575])

In [162]:
l = nn.Linear(1575, 128)

In [163]:
t = l(ajustado.permute(0, 2, 1))

In [166]:
t.permute(0, 2, 1).shape

torch.Size([1, 128, 768])